In [213]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

In [214]:
#provincias = pd.read_csv('../datasets/provicias_datos_2022.csv',
 #                  delimiter=';',
 #                  header=3)

In [215]:
provincias = pd.read_csv('provicias_datos_2022.csv',
                   delimiter=';',
                   header=3)

In [216]:
provincias

,PROVINCIa,población (miles),hogares (miles),ingresos laborales (miles pesos),tasa de empleo (%),superficie (km2),exportaciones (mill usd)
0,Ciudad Autónoma de Buenos Aires,3.004,1.263,211.877.384,"4,7","205,9","311,163"
1,Buenos Aires,17.292,5.738,623.250.917,"8,7","305.907,40","33024,657"
2,Catamarca,336.000,102.000,8.460.986,"7,3","101.486,10","201,03"
3,Chaco,3.477,1.251,122.790.341,"6,5","99.763,30","516,336"
4,Chubut,967.000,287.000,24.733.558,"5,3","224.302,30","2575,372"
5,Córdoba,1.044,321.000,22.744.468,"2,6","164.707,80","12851,979"
6,Corrientes,591.000,205.000,30.606.036,"3,6","89.123,30","273,185"
7,Entre Ríos,1.252,431.000,40.510.249,"6,4","78.383,70","1911,902"
8,Formosa,509.000,148.000,9.586.123,"3,3","75.488,30","40,614"
9,Jujuy,696.000,197.000,17.044.750,"5,1","53.244,20","873,44"


In [217]:
provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PROVINCIa                         24 non-null     object 
 1   población (miles)                 23 non-null     float64
 2   hogares (miles)                   24 non-null     float64
 3   ingresos laborales (miles pesos)  23 non-null     object 
 4   tasa de empleo (%)                24 non-null     object 
 5   superficie (km2)                  24 non-null     object 
 6   exportaciones (mill usd)          24 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.4+ KB


## Limpieza de datos

Notamos que hay un error en el nombre del columna de tasa de empleo, en realidad se trata de la tasa de **desempleo**. Asumimos esto porque no tiene sentido que, por ejemplo, Buenos Aires tenga un $8.7\%$ de empleo registrado.

In [218]:
provincias.rename(
    columns={
        'PROVINCIa': 'provincia',
        'población (miles)': 'poblacion_miles',
        'hogares (miles)': 'hogares_miles',
        'ingresos laborales (miles pesos)': 'ingresos_miles',
        'tasa de empleo (%)': 'tasa_desempleo',
        'superficie (km2)': 'superficie_km2',
        'exportaciones (mill usd)': 'exportaciones_usd_millones'
    },
    inplace=True
)

Multiplicamos por mil ciertas columnas porque en la carga se leyeron como números con decimales, por ejemplo en la primer entrada el campo `poblacion` es `3.004` cuando debería ser `3004`.

Además una de las entradas tiene este campo vacío, por ahora lo asumimos como `0` para poder castear la columna y luego lo imputaremos.

In [219]:
provincias.poblacion_miles = \
    provincias.poblacion_miles\
        .apply(lambda x: x * 1000 if x < 100 else x)\
        .fillna(0).round().astype('int64')

In [220]:
provincias.hogares_miles = \
    provincias.hogares_miles\
        .apply(lambda x: x * 1000 if x < 10 else x)\
        .round().astype('int64')

Una de las entradas no tiene valor en el campo `ingresos_miles`, primero la reemplazamos por `0` para poder castear la columna a tipo entero, después imputaremos ese valor faltante.

In [221]:
provincias.ingresos_miles = \
    provincias.ingresos_miles\
    .str.replace('.', '', regex=False)\
    .fillna(0)\
    .astype('int64')

In [222]:
provincias.tasa_desempleo = \
    provincias.tasa_desempleo\
        .str.replace(',', '.', regex=False)\
        .astype('float64')

Se realizan los reemplazos necesarios en el formato del campo superficie_km2 para realizar el cambio a tipo float.Se redondea a 4 cifras para no perder informacion y luego se mutiplica por mil para obtener el valor correcto  

In [223]:
provincias.superficie_km2 = \
    provincias.superficie_km2\
        .apply(lambda x:
            x.replace(',', '.')
            if '.' not in x
            else x.replace(',', '')
            )\
        .astype('float64').round(4)*1000     

Notamos que la superficie de Tucumán se cargó incorrectamente en la base de datos, por eso vamos a reemplazarlo por el valor obtenido en https://www.argentina.gob.ar/tucuman.

In [224]:
provincias.loc[provincias.provincia == 'Tucumán',
               'superficie_km2'] = 22524

En el caso de las exportaciones decidimos redondearlo a dos decimales.

In [225]:
provincias.exportaciones_usd_millones = \
    provincias.exportaciones_usd_millones\
    .str.replace(',', '.', regex=False)\
    .astype(float).round(2)

Para imputar la población de San Luis decidimos hacer un promedio de las provincias de la región de Cuyo, es decir usaremos el promedio entre las poblaciones de La Rioja, San Juan y Mendoza.

In [226]:
provincias_cuyo = ['La Rioja', 'San Juan', 'Mendoza']
media_cuyo = provincias\
                [provincias.provincia.isin(provincias_cuyo)]\
                .poblacion_miles.mean().astype('int64')

In [227]:
provincias.loc[provincias.provincia == 'San Luis', 'poblacion_miles'] = \
    media_cuyo

Para imputar el ingreso faltante de Río Negro vamos a hacer el promedio de los ingresos de las otras provincias de la región.

In [228]:
provincias_patagonia = ['Neuquén', 'Chubut',
                        'Santa Cruz', 'Tierra del Fuego']

media_patagonia = provincias\
                    [provincias.provincia.isin(provincias_patagonia)]\
                    .ingresos_miles.mean().astype('int64')

In [229]:
provincias.loc[provincias.provincia == 'Río Negro', 'ingresos_miles'] = \
    media_patagonia

In [230]:
provincias_patagonia.remove('Tierra del Fuego')
provincias_patagonia.append('Río Negro')

Imputamos la tasa de desempleo de Tierra del Fuego con el promedio de la misma variable de las otras provincias de la patagonia.

In [231]:
provincias.loc[provincias.provincia == 'Tierra del Fuego', 'tasa_desempleo'] = \
    provincias\
        [provincias.provincia.isin(provincias_patagonia)]\
        .tasa_desempleo.mean()


In [238]:
provincias

,provincia,poblacion_miles,hogares_miles,ingresos_miles,tasa_desempleo,superficie_km2,exportaciones_usd_millones
0,Ciudad Autónoma de Buenos Aires,3004,1263,211877384,4.70,205900.0,311.16
1,Buenos Aires,17292,5738,623250917,8.70,305907.4,33024.66
2,Catamarca,336,102,8460986,7.30,101486.1,201.03
3,Chaco,1044,321,22744468,6.50,99763.3,516.34
4,Chubut,967,287,24733558,5.30,224302.3,2575.37
5,Córdoba,3477,1251,122790341,2.60,164707.8,12851.98
6,Corrientes,1197,205,30606036,3.60,89123.3,273.18
7,Entre Ríos,1252,431,40510249,6.40,78383.7,1911.90
8,Formosa,509,148,9586123,3.30,75488.3,40.61
9,Jujuy,696,197,17044750,5.10,53244.2,873.44


## Exploración de datos

In [233]:
provincias.describe()

,poblacion_miles,hogares_miles,ingresos_miles,tasa_desempleo,superficie_km2,exportaciones_usd_millones
count,24.000000,24.000000,2.400000e+01,24.000000,24.000000,24.000000
mean,1779.958333,593.208333,6.466724e+07,5.081250,124483.525000,3626.671667
std,3428.814581,1152.599519,1.284444e+08,1.694801,72571.430981,7644.733695
min,172.000000,60.000000,8.460986e+06,1.700000,22524.000000,40.610000
25%,570.500000,148.000000,1.589288e+07,3.525000,77659.850000,424.422500
50%,818.500000,236.500000,2.388517e+07,5.200000,100624.700000,1058.325000
75%,1331.000000,410.000000,3.619542e+07,6.325000,157682.325000,1994.452500
max,17292.000000,5738.000000,6.232509e+08,8.700000,305907.400000,33024.660000


Observamos que la poblacion de Chaco comparada con Ciudad de Bs As y Cordoba, tenia un valor muy alto. Verificamos en https://www.argentina.gob.ar y concluimos que los datos estaban intercambiados con Cordoba en los campos poblacion_miles, hogares_miles, ingresos_miles puesto que ciudad de Bs AS y Cordoba deberian tener para un numero de habitantes similar, una cantidad de hogares parecida y los ingresos de Cordoba deben ser mayores a los de Chaco. 

In [234]:
datos_chaco = provincias.iloc[3,0:4]
datos_cordoba = provincias.iloc[5,0:4]

datos_cordoba['provincia'] = 'Chaco'
datos_chaco['provincia'] = 'Córdoba'

provincias.iloc[3,0:4] = datos_cordoba
provincias.iloc[5,0:4] = datos_chaco

In [235]:
provincias_densidad = provincias.poblacion_miles*1000/provincias.superficie_km2
datos_densidad = pd.Series(list(provincias_densidad), index=provincias['provincia'])
datos_densidad

provincia
Ciudad Autónoma de Buenos Aires    14.589607
Buenos Aires                       56.526910
Catamarca                           3.310798
Chaco                              10.464770
Chubut                              4.311146
Córdoba                            21.110111
Corrientes                          6.631263
Entre Ríos                         15.972709
Formosa                             6.742767
Jujuy                              13.071846
La Pampa                            2.209175
La Rioja                            3.890978
Mendoza                            11.189434
Misiones                           33.532366
Neuquén                             6.481540
Río Negro                           3.398154
Salta                               8.413775
San Juan                            7.916535
San Luis                           12.037623
Santa Cruz                          1.550372
Santa Fe                           24.848198
Santiago del Estero                 5.331024


Analizando la densidad de poblacion por provincia, nos resulta bajo el valor resultante de la provincia de Corrientes, por lo que verificamos si poblacion_miles y superficie_km2 estaban correctamente cargadas en https://www.argentina.gob.ar/ y procedemos al cambio de valor de poblacion de dicha provincia, expresado en miles

In [236]:
provincias.loc[provincias.provincia == 'Corrientes',
               'poblacion_miles'] = 1197

 Los complejos soja, maicero, petrolero-petroquímico, automotriz, triguero, carne y cuero bovinos, oro y plata, girasol, pesquero y cebada concentraron el 78,6% del total de las exportaciones argentinas segun el Index en la pagina https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-2-79 corroboramos los datos de nuestro dataset ya que nos intresa estudiar si hay relacion entre superficie, exportaciones y poblacion

## Exportación a formato parquet

In [237]:
tabla = pa.Table.from_pandas(provincias)
pq.write_table(tabla, 'provincias.parquet')

# Conclusiones